In [2]:
import pandas as pd
import numpy as np
import os

In [76]:
class TransactionHistory:
    """
    Class that will separate transaction history by categories
    """
    def __init__(self):
        self.user_in_swap_df = None
        self.user_out_swap_df = None
        self.pool_investments_df = None
        self.pool_extractions_df = None
        self.pool_address = None
        
    
    def classify_history(self, transactions_df: pd.DataFrame):
        """
        Separate history in different dataframes conform transaction parameters
        (into 'in swaps', 'out swaps', investitions into pool and extraction of 
        funds from pool)
        
        Keyword arguments:
        transactions_df (pandas.DataFrame) -- dataframe with all transactions
            in given pool
        """
        self.pool_address = transactions_df['From'].mode()[0]
        
        group_df = transactions_df.groupby('Txhash').agg({
            'From': self.__contract_address_count__,
            'To': self.__contract_address_count__
        })
        
        swap_txhash_list = group_df[(group_df['From'] == 1) & (group_df['To'] == 1)].index.values
        self.user_in_swap_df = transactions_df[(transactions_df['Txhash'].isin(swap_txhash_list)) & (transactions_df['To'] == self.pool_address)]
        self.user_out_swap_df = transactions_df[(transactions_df['Txhash'].isin(swap_txhash_list)) & (transactions_df['From'] == self.pool_address)]
        
        self.pool_investments_df = transactions_df[transactions_df['Txhash'].isin(group_df[group_df['To'] > 1])]
        self.pool_extractions_df = transactions_df[transactions_df['Txhash'].isin(group_df[group_df['From'] > 1])]
        
        
    def __contract_address_count__(self, s):
        """
        inner function that is used for counting directions of funds movement
        into/from pool address
        """
        return (s == self.pool_address).sum()

# WBTC/USDC

In [35]:
print(os.getcwd())

wbtc_df = pd.read_csv(os.getcwd() + "\WBTC-USDT_01-10_31-10.csv", index_col=False)

I:\playground\monte_carlo_sim


In [40]:
print(wbtc_df['From'].mode()[0])
print(wbtc_df['To'].mode()[0])

0x99ac8ca7087fa4a2a1fb6357269965a2014abc35
0x99ac8ca7087fa4a2a1fb6357269965a2014abc35


In [27]:
wbtc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Txhash           5000 non-null   object
 1   UnixTimestamp    5000 non-null   int64 
 2   DateTime         5000 non-null   object
 3   From             5000 non-null   object
 4   To               5000 non-null   object
 5   Value            5000 non-null   object
 6   ContractAddress  5000 non-null   object
 7   TokenName        5000 non-null   object
 8   TokenSymbol      5000 non-null   object
dtypes: int64(1), object(8)
memory usage: 351.7+ KB


In [29]:
wbtc_df.tail(10)

,Txhash,UnixTimestamp,DateTime,From,To,Value,ContractAddress,TokenName,TokenSymbol
4990,0xe4aa44b11fdbedf4606d37b76e8733cb0850c118d576...,1631480075,2021-09-12 20:54:35,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x0000006daea1723962647b7e189d311d757fb793,5.54269751,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,Wrapped BTC,WBTC
4991,0xe4aa44b11fdbedf4606d37b76e8733cb0850c118d576...,1631480075,2021-09-12 20:54:35,0x0000006daea1723962647b7e189d311d757fb793,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,74.9978,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH
4992,0x37e364174382854268efedaed0245547276f27f07200...,1631480075,2021-09-12 20:54:35,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x0000006daea1723962647b7e189d311d757fb793,5.5386518,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,Wrapped BTC,WBTC
4993,0x37e364174382854268efedaed0245547276f27f07200...,1631480075,2021-09-12 20:54:35,0x0000006daea1723962647b7e189d311d757fb793,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,74.9978,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH
4994,0xcb722347f2a370b6252aff73e0a54a59c38100d5fe0b...,1631480299,2021-09-12 20:58:19,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x0000006daea1723962647b7e189d311d757fb793,5.56132999,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,Wrapped BTC,WBTC
4995,0xcb722347f2a370b6252aff73e0a54a59c38100d5fe0b...,1631480299,2021-09-12 20:58:19,0x0000006daea1723962647b7e189d311d757fb793,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,75.36,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH
4996,0xf0796225f1b2442e442b40bec60a8809478ff97acf24...,1631480398,2021-09-12 20:59:58,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x0000006daea1723962647b7e189d311d757fb793,5.54931487,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,Wrapped BTC,WBTC
4997,0xf0796225f1b2442e442b40bec60a8809478ff97acf24...,1631480398,2021-09-12 20:59:58,0x0000006daea1723962647b7e189d311d757fb793,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,75.2523,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH
4998,0xd4be6de779f2c8b71c563119991d7b1fe2987a2b9661...,1631480488,2021-09-12 21:01:28,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,0x27239549dd40e1d60f5b80b0c4196923745b1fd2,3.2874728,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,Wrapped BTC,WBTC
4999,0xd4be6de779f2c8b71c563119991d7b1fe2987a2b9661...,1631480488,2021-09-12 21:01:28,0x27239549dd40e1d60f5b80b0c4196923745b1fd2,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,44.60625,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH


In [71]:
th = TransactionHistory()
th.classify_history(wbtc_df)

In [72]:
th.user_in_swap_df.head(5)

,Txhash,UnixTimestamp,DateTime,From,To,Value,ContractAddress,TokenName,TokenSymbol
2726,0x0023cfe1169713c79fc1dd040f5fbc880253e53af531...,1633896618,2021-10-10 20:10:18,0x0000006daea1723962647b7e189d311d757fb793,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,"149,840.196968",0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC
3263,0x002acd84c85171b00df65f9ab1d8244d787b91db4288...,1634022026,2021-10-12 07:00:26,0xe592427a0aece92de3edee1f18e0157c05861564,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,"58,007.095264",0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC
4067,0x0052150ef1571358dc4884cecafe3bfe44a0d5c2054b...,1634208556,2021-10-14 10:49:16,0xcd9e13b2f3bfc26cd99989fb849a2751932595c4,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,"145,278.931056",0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC
3554,0x005467d1d99cb08ee379ded24e8d31941b7f45b9bc8b...,1634083155,2021-10-12 23:59:15,0xcd9e13b2f3bfc26cd99989fb849a2751932595c4,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,2.71072112,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,Wrapped BTC,WBTC
543,0x005fc1a712571815382ca71f72931b88f719eb50e46d...,1633218999,2021-10-02 23:56:39,0xa4663e48a4e2565d4d7c0ff85321294e2a313cfb,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,"3,436.293926",0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC


In [66]:
th.user_out_swap_df.head(5)

,Txhash,UnixTimestamp,DateTime,From,To,Value,ContractAddress,TokenName,TokenSymbol
2725,0x0023cfe1169713c79fc1dd040f5fbc880253e53af531...,1633896618,2021-10-10 20:10:18,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,0x0000006daea1723962647b7e189d311d757fb793,2.6983,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,Wrapped BTC,WBTC
3262,0x002acd84c85171b00df65f9ab1d8244d787b91db4288...,1634022026,2021-10-12 07:00:26,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,0x7c395dc22e6902b1d8197c2af5290e346ae08a23,1.00923338,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,Wrapped BTC,WBTC
4066,0x0052150ef1571358dc4884cecafe3bfe44a0d5c2054b...,1634208556,2021-10-14 10:49:16,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,0xcd9e13b2f3bfc26cd99989fb849a2751932595c4,2.5164622,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,Wrapped BTC,WBTC
3553,0x005467d1d99cb08ee379ded24e8d31941b7f45b9bc8b...,1634083155,2021-10-12 23:59:15,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,0xcd9e13b2f3bfc26cd99989fb849a2751932595c4,"151,831.696875",0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USD Coin,USDC
542,0x005fc1a712571815382ca71f72931b88f719eb50e46d...,1633218999,2021-10-02 23:56:39,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,0xe592427a0aece92de3edee1f18e0157c05861564,0.07199734,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,Wrapped BTC,WBTC


In [75]:
th.pool_investments_df.head(5)

,Txhash,UnixTimestamp,DateTime,From,To,Value,ContractAddress,TokenName,TokenSymbol


In [74]:
th.pool_extractions_df.head(5)

,Txhash,UnixTimestamp,DateTime,From,To,Value,ContractAddress,TokenName,TokenSymbol
